# IATI Currency Converion with Pandas

In [2]:
import pandas as pd
import numpy as np

In [19]:
# Exchange Rate Data

exchange_df = pd.read_csv("../../data/rates_06_07_2022.csv", dtype={"Rate": np.float64 }, parse_dates=["Date"])

exchange_df.head()

exchange_df[(exchange_df["Date"] == pd.Timestamp('2020-10-01')) & (exchange_df["Currency"] == 'GBP')]

,Date,Currency,Rate
509642,2020-10-01,GBP,1.28365


In [4]:
# Currency Conversion Function
def convert_currency(date, currency, value):
    rates = exchange_df[(exchange_df["Date"] == date) & (exchange_df["Currency"] == currency)]
    if len(rates) > 0:
        exchange_rate = exchange_df.at[rates.index[0], "Rate"]
        if exchange_rate:
            return exchange_rate * value
    return np.NaN

In [14]:
# Transaction Currency Conversion

transaction_df = pd.read_csv("../../data/sample_transactions.csv", dtype={"transaction_value": np.float64})

# get column with currency to use for conversion, first choice is transaction_value_currency, with default_currency fallback
transaction_df["currency_to_use"] = transaction_df["transaction_value_currency"].combine_first(transaction_df["default_currency"])

# get column with date to use for conversion, first choice is transaction_value_value_date, with transaction_transaction_date_iso_date fallback
transaction_df["date_to_use"] = transaction_df["transaction_value_value_date"].combine_first(transaction_df["transaction_transaction_date_iso_date"])

# get only the date part of the date, not the time
def conv_date(date):
    just_date = date.split('T')[0]
    return pd.Timestamp(just_date)

transaction_df["date_to_use"] = transaction_df["date_to_use"].apply(lambda x: conv_date(x))

transaction_df['transaction_value_USD'] = transaction_df.apply(lambda x: convert_currency(date=x["date_to_use"],currency=x["currency_to_use"], value = x["transaction_value"]), axis=1)

transaction_df[["currency_to_use", "date_to_use", "transaction_value", "transaction_value_USD"]]

10


,currency_to_use,date_to_use,transaction_value,transaction_value_USD
0,DKK,2021-11-10,200000.00,1.287000e+06
1,GBP,2020-10-26,510561.88,6.665385e+05
2,GBP,2020-10-01,104965.64,1.347391e+05
3,EUR,2020-12-31,8045.00,9.857270e+03
4,EUR,2015-01-01,210.00,2.540118e+02
5,EUR,2015-12-31,210.00,2.286270e+02
6,USD,2022-01-01,249178.00,2.491780e+05
7,USD,2019-01-01,541059.00,5.410590e+05
8,USD,2020-01-01,8118514.00,8.118514e+06
9,USD,2021-01-01,3599264.00,3.599264e+06


In [23]:
# Transaction Tests

assert transaction_df["transaction_value"].iloc[2] == 104965.64, "transaction_value 3rd row should be 104965.64"
assert transaction_df["transaction_value_USD"].iloc[2] == 104965.64 * 1.28365 , "transaction_value_USD 3rd row should be 104965.64 * 1.28365"

assert transaction_df["transaction_value"].iloc[6] == transaction_df["transaction_value_USD"].iloc[6], "transaction_value for USD row 7 should equal transaction_value_USD"
